In [1]:
from qdrant_client import QdrantClient, models
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
client = QdrantClient(url=os.getenv("QDRANT_URL"), api_key=os.getenv("QDRANT_API_KEY"))

In [5]:
if not client.collection_exists(collection_name = collection_name):
    client.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=4,
            distance=models.Distance.COSINE
        )
    )

In [6]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='first_practice'), CollectionDescription(name='first_lesson')])

In [7]:
points = [
    models.PointStruct(
        id=1,
        vector=[0.1, 0.2, 0.3, 0.4],
        payload={"category":"example"}
    ),
    models.PointStruct(
        id=2,
        vector=[0.2, 0.3, 0.4, 0.5],
        payload={"category":"demo"}
    )
]

client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
collection_info = client.get_collection(collection_name)
collection_info

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, warnings=None, indexed_vectors_count=0, points_count=2, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_a

In [9]:
query_vector = [0.08, 0.14, 0.33, 0.28]

search_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=1
)

print(search_results)

points=[ScoredPoint(id=1, version=2, score=0.97642946, payload={'category': 'example'}, vector=None, shard_key=None, order_value=None)]


In [12]:
new_collection = "first_practice"

if not client.collection_exists(collection_name=new_collection):
    client.create_collection(
        collection_name=new_collection,
        vectors_config=models.VectorParams(
            size=4,
            distance=models.Distance.COSINE
        )
    )

In [13]:
client.create_payload_index(
    collection_name=new_collection,
    field_name="category",
    field_schema=models.PayloadSchemaType.KEYWORD
)

UpdateResult(operation_id=4, status=<UpdateStatus.COMPLETED: 'completed'>)

Here each dimension has a meaning, for example: <br>
The 0th: affordability <br>
1th: quality <br>
2th: popularity <br>
3th: innovation

In [30]:
points=[
    models.PointStruct(
        id=1,
        vector=[0.6, 0.1, 0.1, 0.8], # Mediun affordability, high innovation
        payload={"name": "Budget Smartphone", "category": "electronics", "price": 299},
    ),
    models.PointStruct(
        id=2,
        vector=[0.2, 0.9, 0.8, 0.5], # High quality, high popularity
        payload={"name": "Bestselling Novel", "category": "books", "price": 19},
    ),
    models.PointStruct(
        id=3,
        vector=[0.8, 0.3, 0.2, 0.9], # High affordability, high innovation (similar to ID 1)
        payload={"name": "Smart Home Hub", "category": "electronics", "price": 89},
    ),
    models.PointStruct(
        id=4,
        vector=[1, 0.7, 0.2, 0.3],
       payload={"name": "Dumbphone", "category": "electronics", "price": 100},
    ),models.PointStruct(
        id=5,
        vector=[0.2, 0.1, 0.1, 0.5],
        payload={"name": "Worstselling Novel", "category": "books", "price": 1},
    )
]

client.upsert(collection_name=new_collection, points=points)


UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

In [31]:
query_vector = [0.9, 0.5, 0.14, 0.4]

basic_results = client.query_points(new_collection, query = query_vector)

filtered_results = client.query_points(
    new_collection,
    query=query_vector,
    query_filter=models.Filter(
        must=[models.FieldCondition(key="category", match=models.MatchValue(value="electronics"))]
    ),
)
print("Filtered result:", filtered_results)

Filtered result: points=[ScoredPoint(id=4, version=5, score=0.9865267, payload={'name': 'Dumbphone', 'category': 'electronics', 'price': 100}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=5, score=0.89889985, payload={'name': 'Smart Home Hub', 'category': 'electronics', 'price': 89}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=5, score=0.82173365, payload={'name': 'Budget Smartphone', 'category': 'electronics', 'price': 299}, vector=None, shard_key=None, order_value=None)]
